<a href="https://colab.research.google.com/github/rfdornelles/mds_ML_project/blob/main/scenarios_xgboost_karon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/rfdornelles/mds_ML_project/main/data/karon2.csv")

In [2]:
df.head()

,year,country,temp,population,qnt_death_heat_cold_exposure,temp_diff
0,1991,Albania,6.219891,3266790,5,0.050951
1,1992,Albania,6.284930,3247039,5,0.065040
2,1993,Albania,6.324316,3227287,5,0.039385
3,1994,Albania,6.357706,3207536,5,0.033390
4,1995,Albania,6.402805,3187784,6,0.045099


In [3]:
X = df.drop(["qnt_death_heat_cold_exposure", "country"], axis = 1).to_numpy()
y = df["qnt_death_heat_cold_exposure"].to_numpy()

In [4]:
#https://www.kaggle.com/code/stuarthallows/using-xgboost-with-scikit-learn/notebook 

In [5]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score

import xgboost as xgb

In [6]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [7]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [8]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

xgb_model.fit(X, y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(y, y_pred)

print(np.sqrt(mse))
print("R2:", r2_score(y, y_pred))

69.66316123128917
R2: 0.9851849903645188


In [9]:
xgb_model

XGBRegressor(objective='reg:squarederror', random_state=42)

In [10]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=13
)

scores = []

In [11]:
for train_index, test_index in kfold.split(X):   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb_model = xgb.XGBRegressor(objective="reg:squarederror")
    xgb_model.fit(X_train, y_train)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred))
    
display_scores(np.sqrt(scores))
print("R2:", r2_score(y_test, y_pred))

Scores: [104.84821268  84.81228395 101.41978451 122.59935431 121.99012335]
Mean: 107.134
Std: 14.114
R2: 0.944357828739073


In [12]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

scores = cross_val_score(xgb_model, X, y, scoring="neg_mean_squared_error", cv=5)

display_scores(np.sqrt(-scores))

Scores: [ 256.46676639 1063.11597536  172.28719001   95.05281799 1210.78643142]
Mean: 559.542
Std: 476.503


In [13]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X, y)

report_best_scores(search.cv_results_, 1)


Fitting 3 folds for each of 200 candidates, totalling 600 fits
Model with rank: 1
Mean validation score: -0.431 (std: 0.314)
Parameters: {'colsample_bytree': 0.7212822750999782, 'gamma': 0.1983919136069442, 'learning_rate': 0.04523055931181908, 'max_depth': 4, 'n_estimators': 122, 'subsample': 0.6110467087494819}



In [14]:
mse = mean_squared_error(y_test, search.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

The mean squared error (MSE) on test set: 14739.6230


In [15]:
#print("coef:", pipe['model'].coef_)
#print("intercept:", pipe['model'].intercept_)
print("R2:", r2_score(y_test, search.predict(X_test)))

R2: 0.9448886432505905


In [16]:
best_params = {'colsample_bytree': 0.9040922615763338, 'gamma': 0.2252496259847715, 'learning_rate': 0.033979488347959955, 'max_depth': 2, 'n_estimators': 113, 'subsample': 0.9233589392465844}

xgb_model = xgb.XGBRegressor(objective="reg:squarederror", 
                             random_state=42, 
                             params = best_params)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse=mean_squared_error(y_test, y_pred)

print(np.sqrt(mse))
print("R2:", r2_score(y_test, y_pred))



121.99012335270251
R2: 0.944357828739073


In [17]:
# Predictions

df_rcp45 = pd.read_csv("https://github.com/rfdornelles/mds_ML_project/raw/main/data/pred_y_2022_2050_rcp45.csv")

X_rcp45 = df_rcp45[df_rcp45["country"] == "Germany"].drop(["country"],axis = 1)

In [18]:
y_rcp45 = xgb_model.predict(X_rcp45.to_numpy())

In [19]:
X_rcp45["pred_rcp45"] = y_rcp45.round()

In [20]:
print(X_rcp45)

     year      temp  population  temp_diff  pred_rcp45
448  2023  7.030051    82837000   0.039303       166.0
449  2024  7.060947    82678000   0.030897       166.0
450  2025  7.105634    82553000   0.044687       173.0
451  2026  7.138498    82460000   0.032863       166.0
452  2027  7.167267    82390000   0.028770       166.0
453  2028  7.177644    82333000   0.010377       155.0
454  2029  7.200059    82276000   0.022415       166.0
455  2030  7.215023    82210000   0.014964       155.0
456  2031  7.242563    82133000   0.027540       166.0
457  2032  7.249095    82045000   0.006532       155.0
458  2033  7.249983    81947000   0.000888       155.0
459  2034  7.290183    81839000   0.040199       166.0
460  2035  7.308158    81720000   0.017975       166.0
461  2036  7.356454    81591000   0.048296       165.0
462  2037  7.373711    81450000   0.017257       155.0
463  2038  7.384128    81295000   0.010417       155.0
464  2039  7.418958    81128000   0.034830       166.0
465  2040 

RCP 85

In [21]:
# Predictions

df_rcp85 = pd.read_csv("https://github.com/rfdornelles/mds_ML_project/raw/main/data/pred_y_2022_2050_rcp85.csv")

X_rcp85 = df_rcp85[df_rcp85["country"] == "Germany"].drop(["country"],axis = 1)

In [22]:
y_rcp85 = xgb_model.predict(X_rcp85.to_numpy())

In [23]:
X_rcp85["pred_rcp85"] = y_rcp85.round()

In [24]:
print(X_rcp85)

     year      temp  population  temp_diff  pred_rcp85
504  2023  7.139563    82837000   0.027236       166.0
505  2024  7.171224    82678000   0.031660       166.0
506  2025  7.221347    82553000   0.050123       165.0
507  2026  7.255625    82460000   0.034278       166.0
508  2027  7.270194    82390000   0.014570       155.0
509  2028  7.305102    82333000   0.034907       166.0
510  2029  7.326372    82276000   0.021270       166.0
511  2030  7.330444    82210000   0.004072       155.0
512  2031  7.347063    82133000   0.016619       155.0
513  2032  7.386325    82045000   0.039262       166.0
514  2033  7.404685    81947000   0.018360       166.0
515  2034  7.405402    81839000   0.000717       155.0
516  2035  7.426229    81720000   0.020827       166.0
517  2036  7.467698    81591000   0.041468       166.0
518  2037  7.517649    81450000   0.049951       165.0
519  2038  7.553091    81295000   0.035442       166.0
520  2039  7.591253    81128000   0.038162       166.0
521  2040 

In [25]:
plot_df = pd.DataFrame()
plot_df["year"] = range(1991, 2051)

In [26]:
df_actual_values = df[df["country"] == "Germany"][["year", "qnt_death_heat_cold_exposure"]]

In [27]:
df_rcp85 = X_rcp85[["year", "pred_rcp85"]]

In [28]:
df_rcp45 = X_rcp45[["year", "pred_rcp45"]]

In [29]:
plot_df = plot_df.merge(df_actual_values, on = "year", how = "left")

plot_df = plot_df.merge(X_rcp45, on = "year", how = "left")

plot_df = plot_df.merge(X_rcp85, on = "year", how = "left")

plot_df

,year,qnt_death_heat_cold_exposure,temp_x,population_x,temp_diff_x,pred_rcp45,temp_y,population_y,temp_diff_y,pred_rcp85
0,1991,204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992,204.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993,211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1994,211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1995,225.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1996,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1997,221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1998,202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1999,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,179.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
